Under development

In [1]:
import pandas as pd

data = pd.read_pickle('event_data.pickle')
data.head()

,time,event,player_sub,main_player,commentary
0,89',Yellow Card,na,K. Phillips,A hasty challenge from Kalvin Phillips now and...
1,87',Substitution,H. Kane,Carlos Vinícius,Harry Kane - who became a father this week - m...
2,85',Yellow Card,na,P. Højbjerg,"Hojbjerg picks up a late booking here, as the ..."
3,78',Substitution,T. Ndombèlé,Lucas Moura,"Tottenham make their second change now, with N..."
4,76',Substitution,H. Winks,M. Sissoko,Lloris gets a glance to this one and Ayling's ...


In [2]:
from process_data import Data_Processor

In [47]:
data_proc = Data_Processor(data['commentary'], 100)

In [4]:
data_proc.vocab_size

27

In [48]:
from sklearn.model_selection import train_test_split

seq = data_proc.seq
targets = data_proc.targets
seq_tr, seq_te, target_tr, target_te = train_test_split(seq[:100000], targets[:100000], 
                                                        test_size=0.33, random_state=42)

In [ ]:
args = {'batch_size' : 256,
        'hidden_dim' : 128,
        'window' : 100,
        'init_method' : 'kaiming'}
num_epochs = 50
"""
    Train the model
"""
from model import TextGen
import torch 

gen = TextGen(args, data_proc.vocab_size)

optimizer = torch.optim.RMSprop(gen.parameters(), lr=0.001)

batches = int(len(seq_tr) / args['batch_size'])
print(batches)

gen.train()

for epoch in range(num_epochs):
    
    for i in range(batches):
        
        try :
            x_batch = seq_tr[i * args['batch_size'] : (i+1) * args['batch_size']]
            y_batch = target_tr[i * args['batch_size'] : (i+1) * args['batch_size']]
        except:
            x_batch = seq_tr[i * args['batch_size'] :]
            y_batch = target_tr[i * args['batch_size'] :]
            
        x = torch.from_numpy(x_batch).type(torch.LongTensor)
        y = torch.from_numpy(y_batch).type(torch.LongTensor)
#         print(x.shape, y.shape)
        y_pred = gen(x)
        
        loss = torch.nn.functional.cross_entropy(y_pred, y.squeeze())
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
        if i == batches - 1 :
            print("Epoch : {}, loss : {:.4f}".format(epoch, loss.item()))

261
Epoch : 0, loss : 2.8287
Epoch : 1, loss : 2.8289
Epoch : 2, loss : 2.8241
Epoch : 3, loss : 2.8206
Epoch : 4, loss : 2.8143
Epoch : 5, loss : 2.8054
Epoch : 6, loss : 2.7936
Epoch : 7, loss : 2.7861
Epoch : 8, loss : 2.7724
Epoch : 9, loss : 2.7525
Epoch : 10, loss : 2.7463
Epoch : 11, loss : 2.6804
Epoch : 12, loss : 2.6960
Epoch : 13, loss : 2.6282


In [ ]:
torch.save(gen.state_dict(), 'weights/textgen_model2.pt')

In [46]:
import numpy as np

gen.eval()

softmax = torch.nn.Softmax(dim=1)

start = np.random.randint(0, len(seq_te)-1)
print(start)
pattern = seq_te[start]

print("\Pattern: \n")
print(''.join([data_proc.idx2ch[value] for value in pattern]), "\"")

full_pred = pattern.copy()

for i in range(100):
    pattern = torch.from_numpy(pattern).type(torch.LongTensor)
    pattern = pattern.view(1,-1)
    
    prediction = gen(pattern)
    prdiction = softmax(prediction)
    
    prediction = prediction.squeeze().detach().numpy()
    
    arg_max = np.argmax(prediction)
    
    pattern = pattern.squeeze().detach().numpy()
    pattern = pattern[1:]
    
    pattern = np.append(pattern, arg_max)
    
    full_pred = np.append(full_pred, arg_max)

print(''.join([data_proc.idx2ch[val] for val in full_pred]), "\"")

6405
\Pattern: 

nogbonna i "
nogbonna i                                                                                                     "
